- 데이터 전처리 하는 선까지 하고 쓸데 없는거 다 날리고 숫자데이터들 뽑기

- 있는 수 데이터로 머신러닝돌리기

- RMSE, Rsquare 등을 뽑아서 값을 뽑아보기

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import warnings
warnings.filterwarnings('ignore') # 워닝 메세지를 생략해줌
import os

- 디렉토리에 data라는 폴더가 있는지 검색

In [2]:
os.listdir("data")

['.DS_Store', 'advertising.csv', 'tvmarketing.csv', '서울_연립다세대_매매']

In [3]:
import os

path = 'data/서울_연립다세대_매매/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝이 .csv인 경우

In [4]:
df = pd.DataFrame()
try:
    for i in file_list_py:
        data = pd.read_csv(path + i, skiprows=15, encoding='cp949', thousands=',')
        df = pd.concat([df,data])
except:
    print('Error!!!')
else:
    print('완료')   
df = df.reset_index(drop = True)

완료


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493355 entries, 0 to 493354
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       493355 non-null  object 
 1   번지        493321 non-null  object 
 2   본번        493355 non-null  int64  
 3   부번        493355 non-null  int64  
 4   건물명       493355 non-null  object 
 5   전용면적(㎡)   493355 non-null  float64
 6   대지권면적(㎡)  493355 non-null  float64
 7   계약년월      493355 non-null  int64  
 8   계약일       493355 non-null  int64  
 9   거래금액(만원)  493355 non-null  int64  
 10  층         493355 non-null  int64  
 11  건축년도      493334 non-null  float64
 12  도로명       493355 non-null  object 
 13  해제사유발생일   4464 non-null    float64
dtypes: float64(4), int64(6), object(4)
memory usage: 52.7+ MB


In [6]:
df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
0,서울특별시 강남구 개포동,170-18,170,18,(170-18),26.60,21.53,201007,13,29350,1,1988.0,선릉로12길 17,NaN
1,서울특별시 강남구 개포동,1219-11,1219,11,강남하이츠,39.90,19.33,201001,22,21500,1,1993.0,개포로20길 35,NaN
2,서울특별시 강남구 개포동,1202-2,1202,2,경모하이츠,51.69,21.20,201003,12,28500,1,1991.0,개포로22길 41-6,NaN
3,서울특별시 강남구 개포동,1235-16,1235,16,광현빌라,62.76,34.53,201001,22,36500,2,2001.0,논현로18길 10-13,NaN
4,서울특별시 강남구 개포동,1235-16,1235,16,광현빌라,77.34,42.55,201011,2,44200,4,2001.0,논현로18길 10-13,NaN


In [7]:
# 쓸 데 없는 컬럼 삭제
df = df.drop('해제사유발생일', axis=1)

In [8]:
df.columns

Index(['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '대지권면적(㎡)', '계약년월', '계약일',
       '거래금액(만원)', '층', '건축년도', '도로명'],
      dtype='object')

In [9]:
df['동_건물명'] = df['시군구'].str.split(' ', expand=True)[1] + '_' + df['건물명']

In [10]:
df = df.rename(columns={'거래금액(만원)': '거래금액',
                       '전용면적(㎡)': '전용면적',
                       '대지권면적(㎡)': '대지권면적'})
df['거래금액'] = df['거래금액'] * 10000

In [12]:
df['동_도로명'] = df['시군구'].str.split(' ', expand=True)[1] + '_' + df['도로명']

In [13]:
df['공급면적'] = df['전용면적'] + df['대지권면적']
df['매물구분'] = df['동_도로명'] + '_' + df['전용면적'].astype('str') # 전용면적은 층마다 다를 수 있다 그래서 공급면적이 아니라 전용면적으로 구분 지음
df['매물구분'].nunique()

210914

In [14]:
df['용적률'] = (df['전용면적'] / df['공급면적']) * 100
df.head()

,시군구,번지,본번,부번,건물명,전용면적,대지권면적,계약년월,계약일,거래금액,층,건축년도,도로명,동_건물명,동_도로명,공급면적,매물구분,용적률
0,서울특별시 강남구 개포동,170-18,170,18,(170-18),26.60,21.53,201007,13,293500000,1,1988.0,선릉로12길 17,강남구_(170-18),강남구_선릉로12길 17,48.13,강남구_선릉로12길 17_26.6,55.266985
1,서울특별시 강남구 개포동,1219-11,1219,11,강남하이츠,39.90,19.33,201001,22,215000000,1,1993.0,개포로20길 35,강남구_강남하이츠,강남구_개포로20길 35,59.23,강남구_개포로20길 35_39.9,67.364511
2,서울특별시 강남구 개포동,1202-2,1202,2,경모하이츠,51.69,21.20,201003,12,285000000,1,1991.0,개포로22길 41-6,강남구_경모하이츠,강남구_개포로22길 41-6,72.89,강남구_개포로22길 41-6_51.69,70.915078
3,서울특별시 강남구 개포동,1235-16,1235,16,광현빌라,62.76,34.53,201001,22,365000000,2,2001.0,논현로18길 10-13,강남구_광현빌라,강남구_논현로18길 10-13,97.29,강남구_논현로18길 10-13_62.76,64.508171
4,서울특별시 강남구 개포동,1235-16,1235,16,광현빌라,77.34,42.55,201011,2,442000000,4,2001.0,논현로18길 10-13,강남구_광현빌라,강남구_논현로18길 10-13,119.89,강남구_논현로18길 10-13_77.34,64.509133


In [15]:
df['최종거래월'] = df.groupby(['매물구분'])['계약년월'].shift(1)
df['최종거래월_월'] = df['계약년월'] - df['최종거래월']

In [16]:
df['최종거래월'] = df.groupby(['매물구분'])['계약년월'].shift(1)
df['최종거래월_월'] = df['계약년월'] - df['최종거래월']

In [ ]:
df['최종거래가_1'] = df['거래금액'] / df['전용면적']
df['최종거래가_1'] = df.groupby(['매물구분'])['최종거래가_1'].shift(1)

df['최종거래가_3'] = df['거래금액'] / df['전용면적']
df['최종거래가_3'] = df.groupby(['매물구분'])['최종거래가_3'].shift(1)

df['최종면적'] = df.groupby(['매물구분'])['계약년월']


In [ ]:
df.isna().sum()

In [ ]:
df.drop(['번지', '본번', '부번', '건물명', '전용면적', '대지권면적', '계약년월', '계약일', '건축년도', '도로명', '매물구분', '최종거래월'], axis=1)
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['구'] = df['시군구'].str.split(' ', expand=True)[1]

In [ ]:
df['구'].value_counts()

In [ ]:
df_cat = pd.get_dummies(df['구'], drop_first=True)

In [ ]:
df.columns

In [ ]:
df = df.drop(['시군구', '번지', '본번', '부번', '건물명', '도로명', '동_건물명', '동_도로명', '공급면적', '매물구분', '구'], axis=1)

In [ ]:
df = pd.concat([df, df_cat], axis=1)

In [ ]:
df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt
corr = df.corr(method='pearson')
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')
plt.rc('font', family = 'Malgun Gothic')
sns.heatmap(corr,annot=False, cmap = 'coolwarm')
plt.show()

In [ ]:
X = df.drop('거래금액', axis=1)
y = df['거래금액']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.3, random_state = 10)
print(X_train)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(len(X_train) + len(X_test))

In [ ]:
y_train = np.log1p(y_train)
sns.distplot(y_train)

In [ ]:
from xgboost import XGBRegressor
from sklearn import metrics

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)